## 1. 视频转换图片

In [3]:
VIDEO_PATH = 'D:/0LT/chitanda/vedio/05.mkv' # 视频地址
EXTRACT_FOLDER = 'D:/0LT/chitanda/vedio_pic/' # 存放帧图片的位置
EXTRACT_FREQUENCY = 100 # 帧提取频率

def extract_frames(video_path, dst_folder, index):
    # 主操作
    import cv2
    video = cv2.VideoCapture()
    if not video.open(video_path):
        print("can not open the video")
        exit(1)
    count = 1
    while True:
        _, frame = video.read()
        if frame is None:
            break
        if count % EXTRACT_FREQUENCY == 0:
            save_path = "{}/{:>04d}.jpg".format(dst_folder, index)
            cv2.imwrite(save_path, frame)
            index += 1
        count += 1
    video.release()
    # 打印出所提取帧的总数
    print("Totally save {:d} pics".format(index-1))
    
if __name__ == '__main__':
    extract_frames(VIDEO_PATH, EXTRACT_FOLDER, 1497)


Totally save 1909 pics


## 2. 手动将不需要的图片删除，修改整理好后的图片名称（有序排列）

rename()不能修改为已经有的名字，会报错
当文件夹内有新旧（打过标签和未打标签）图片时，排序并非看起来的数字排序，所以可能未达标签的顺序可能比较靠前，使得重新命名的时候会命名为
已经存在的名字，从而报错。
所以最好将打过标签的图片存放在别的文件夹内，等这边的修改完再考进来（此时修改要记住命名的起点不是1）。

In [15]:
import os  
      
class BatchRename():  
        #批量重命名文件夹中的图片文件 
        def __init__(self):  
            self.path = 'D:/0LT/chitanda/VOC2007/JPEGImages/'  #图片文件夹路径
      
        def rename(self):  
            filelist = os.listdir(self.path)  
            total_num = len(filelist)  
            i = 816  #从1开始排序
            for item in filelist:  
                if item.endswith('.jpg'):  
                    n = 3 - len(str(i))  
                    src = os.path.join(os.path.abspath(self.path), item)  
                    dst = "{}{:03d}.jpg".format(self.path, i) 
                    try:  
                        os.rename(src, dst)  
                        i = i + 1 
                    except:  
                        print(src,dst)
                        break  
            print('total %d to rename & converted %d jpgs' % (total_num, i))  
      
if __name__ == '__main__':  
        demo = BatchRename()  
        demo.rename()  

total 42 to rename & converted 858 jpgs


## 3. labelimg给图片打上标签 

## 4. 划分数据集

In [1]:
import os
import random

xmlfilepath = r'F:/Traing_folderV2/muck_car/Annotations/'
saveBasePath = r'F:/Traing_folderV2/muck_car/ImageSets/'

trainval_percent = 0.9
train_percent = 0.9
total_xml = os.listdir(xmlfilepath)
num = len(total_xml)
list = range(num)
tv = int(num * trainval_percent)
tr = int(tv * train_percent)
trainval = random.sample(list, tv)
train = random.sample(trainval, tr)

print("train and val size", tv)
print("train size", tr)
print('test sizr',num-tv)
ftrainval = open(os.path.join(saveBasePath, 'Main/trainval.txt'), 'w')
ftest = open(os.path.join(saveBasePath, 'Main/test.txt'), 'w')
ftrain = open(os.path.join(saveBasePath, 'Main/train.txt'), 'w')
fval = open(os.path.join(saveBasePath, 'Main/val.txt'), 'w')

for i in list:
    name = total_xml[i][:-3] + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

train and val size 3081
train size 2772
test sizr 343


## 4. 将图像数据修改tf格式

### 4.0 修改SSD-Tensorflow/datasets/pascalvoc_to_tfrecodes.py 的读取格式：

### 4.1 按照自己的数据集类型修改文件：SSD-Tensorflow/datasets/pascalvoc_common.py，示例如下： 

### 4.2 新建tfrecords_文件，保存tf格式的数据

### 4.3 在pycharm里面运行tf_convert_data.py，输入如下参数.第一行和第三行不用更改，二四两行要改成绝对地址。

In [1]:
%run tf_convert_data.py \
--dataset_name=pascalvoc \
--dataset_dir=F:/Traing_folderV2/muck_car/ \
--output_name=voc_2007_train \
--output_dir=F:/Traing_folderV2/muck_car/tfrecords_/

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Dataset directory: F:/Traing_folderV2/muck_car/
Output directory: F:/Traing_folderV2/muck_car/test/
>> Converting image 1/3424WARNING:tensorflow:From F:\Traing_folderV2\SSD-Tensorflow-muck_car\datasets\pascalvoc_to_tfrecords.py:83: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
>> Converting image 3424/3424
Finished converting the Pascal VOC dataset!


## 5.训练模型 

### 5.1 统计数据集的一些信息

In [1]:
import re
import os
import xml.etree.ElementTree as ET

Class = ['muck_car','bus','truck','trailer']
annotation_folder =  r'F:/Traing_folderV2/muck_car/Annotations/'  # 改为自己标签文件夹的路径
list = os.listdir(annotation_folder)
def file_name(file_dir):
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.xml':
                L.append(os.path.join(root, file))
    return L
total_Box = 0;total_Pic = 0
box_num=[0 for i in range(len(Class))]
pic_num=[0 for i in range(len(Class))]
flag=[0 for i in range(len(Class))]

xml_dirs = file_name(annotation_folder)

for i in range(0, len(xml_dirs)):
    #print(xml_dirs[i])
    annotation_file = open(xml_dirs[i]).read()
    root = ET.fromstring(annotation_file)
    total_Pic += 1
    for obj in root.findall('object'):
        label = obj.find('name').text
        for i in range(len(Class)):
            if label == Class[i]:
                box_num[i] += 1
                flag[i] = 1
                total_Box += 1
    for i in range(len(Class)):
        if flag[i] == 1:
            pic_num[i] += 1
            flag[i] = 0

for i in range(len(Class)):
    print(Class[i], pic_num[i], box_num[i])
print("total", total_Pic, total_Box)

muck_car 2346 3109
bus 1370 1679
truck 1198 1418
trailer 418 418
total 3424 6624


### 5.2 按照上面的结果修改pascalvoc_2007.py

### 5.3 修改ssd_vgg_300.py     96，97行

### 5.4 修改eval_ssd_network.py    66行

### 5.5 修改 train_ssd_network.py     35，142，50-74，158，162 行

## 6.训练模型

同上，在pycharm里面运行，train_ssd_network.py 输入如下参数，修改地址即可，其它酌情修改。 

### 6.1从零开始

#备注 batch_size不要过大，之前为32，会报错GPU内存不够，现改为8

### 6.2加载现有模型

# 7.验证

### *SSD原版验证代码需要解决以下问题：
####     1.tf版本过高：
#####       不用降低版本：./tf_extended/metrics.py中 51行，"return variables.Variable" =>> "return variables.VariableV1"

#### 2.缺失flatten：
##### 定义flatten()并修改327 和 346行：eval_op=list(...) => eval_op=flatten(list(...))；flatten()如下

## 7.1 生成验证数据集

In [1]:
%run tf_convert_data.py \
--dataset_name=pascalvoc \
--dataset_dir=F:/Traing_folderV2/test/ \
--output_name=voc_2007_test \
--output_dir=F:/Traing_folderV2/test/tfrecords_/

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Dataset directory: F:/Traing_folderV2/test/
Output directory: F:/Traing_folderV2/test/tfrecords_/
>> Converting image 1/116WARNING:tensorflow:From F:\Traing_folderV2\SSD-Tensorflow-muck_car\datasets\pascalvoc_to_tfrecords.py:83: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
>> Converting image 116/116
Finished converting the Pascal VOC dataset!


## 7.2 运行验证代码

In [1]:
%run eval_ssd_network.py \
--eval_dir=F:/Traing_folderV2/SSD-Tensorflow-muck_car/eval_log/ \
--dataset_dir=F:/Traing_folderV2/test/tfrecords_/ \
--dataset_name=pascalvoc_2007 \
--dataset_split_name=test \
--model_name=ssd_512_vgg \
--checkpoint_path=F:/Traing_folderV2/SSD-Tensorflow-muck_car/checkpoints/model.ckpt-79239 \
--batch_size=1 \
--select_threshold=0.8 \
--nms_threshold=0.1 \
--matching_threshold=0.6

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Colocations handled automatically by placer.

# =========================================================================== #
# Training | Evaluation flags:
# =========================================================================== #
{'batch_size': <absl.flags._flag.Flag object at 0x000000001108A8C8>,
 'checkpoint_path': <absl.flags._flag.Flag object at 0x000000001108AF88>,
 'dataset_dir': <absl.flags._flag.Flag object at 0x000000001108D4C8>,
 'dataset_name': <absl.flags._flag.Flag object at 0x000000001108D2C8>,
 'dataset_split_name': <absl.flags._flag.Flag object at 0x000000001108D3C8>,
 'eval_dir': <absl.flags._flag.Flag object at 0x000000001108D0C8>,
 'eval_image_size': <absl.flags._flag.Flag object at 0x000000001108AA88>,
 'eval_resize': <absl.flags._flag.Flag object at 0x000000001108A988>,
 'gpu_memory_fraction': <absl.flags._flag.Flag object at 0x000000001108D808>,
 'h': <

In [10]:
%time
s=99999999999*99999
print(s)

Wall time: 0 ns
9999899999900001
